In [2]:
import pandas as pd
from tkinter import Tk, filedialog
import os

def browse_file(title="Select a file"):
    """Open file browser dialog to select a file."""
    root = Tk()
    root.withdraw()
    root.attributes('-topmost', True)
    file_path = filedialog.askopenfilename(
        title=title,
        filetypes=[("CSV files", "*.csv"), ("Excel files", "*.xlsx"), ("All files", "*.*")]
    )
    root.destroy()
    return file_path

def clean_ticker(ticker_str):
    """Extract just the ticker symbol from strings like 'LLY UN Equity'"""
    if pd.isna(ticker_str):
        return ''
    return str(ticker_str).strip().split()[0]

def check_tickers(input_file, reference_file):
    """
    Checks if tickers from input file exist in reference file.
    
    Parameters:
    - input_file: Path to CSV file with tickers to check
    - reference_file: Path to CSV file with reference tickers (e.g., WLS index)
    """
    
    # Read the files
    print(f"Reading input file: {os.path.basename(input_file)}")
    if input_file.endswith('.xlsx'):
        input_df = pd.read_excel(input_file)
    else:
        input_df = pd.read_csv(input_file)
    
    print(f"Reading reference file: {os.path.basename(reference_file)}")
    if reference_file.endswith('.xlsx'):
        reference_df = pd.read_excel(reference_file)
    else:
        reference_df = pd.read_csv(reference_file)
    
    # Get first column for tickers
    input_col = input_df.columns[0]
    reference_col = reference_df.columns[0]
    
    print(f"Input ticker column: {input_col}")
    print(f"Reference ticker column: {reference_col}")
    
    # Clean tickers from input file
    input_df['Original_Ticker'] = input_df[input_col]
    input_df['Clean_Ticker'] = input_df[input_col].apply(clean_ticker)
    
    # Clean reference tickers and convert to uppercase set
    reference_tickers = set(
        reference_df[reference_col].apply(clean_ticker).str.upper()
    )
    
    # Check matches (case-insensitive)
    input_df['In_Reference'] = input_df['Clean_Ticker'].str.upper().isin(reference_tickers)
    input_df['Status'] = input_df['In_Reference'].apply(
        lambda x: 'Found in Reference' if x else 'NOT in Reference'
    )
    
    # Create summary results
    matched = input_df[input_df['In_Reference'] == True]
    not_matched = input_df[input_df['In_Reference'] == False]
    
    # Save main results file
    results_file = 'ticker_comparison_results.csv'
    results_df = input_df[['Original_Ticker', 'Clean_Ticker', 'In_Reference', 'Status']]
    results_df.to_csv(results_file, index=False)
    
    # Save matched tickers (same format as input)
    matched_file = 'tickers_MATCHED.csv'
    matched_output = pd.DataFrame({
        'Ticker': matched['Original_Ticker']
    })
    matched_output.to_csv(matched_file, index=False)
    
    # Save not matched tickers (same format as input)
    not_matched_file = 'tickers_NOT_MATCHED.csv'
    not_matched_output = pd.DataFrame({
        'Ticker': not_matched['Original_Ticker']
    })
    not_matched_output.to_csv(not_matched_file, index=False)
    
    # Print summary
    print("\n" + "="*60)
    print("SUMMARY")
    print("="*60)
    print(f"Total tickers in input file: {len(input_df)}")
    print(f"Tickers found in reference: {len(matched)} ({len(matched)/len(input_df)*100:.1f}%)")
    print(f"Tickers NOT found in reference: {len(not_matched)} ({len(not_matched)/len(input_df)*100:.1f}%)")
    
    print("\n" + "="*60)
    print("FILES CREATED:")
    print("="*60)
    print(f"1. {results_file} - Full comparison results")
    print(f"2. {matched_file} - Tickers found in reference ({len(matched)} tickers)")
    print(f"3. {not_matched_file} - Tickers NOT in reference ({len(not_matched)} tickers)")
    
    if len(not_matched) > 0:
        print("\n" + "-"*60)
        print(f"TICKERS NOT IN REFERENCE FILE ({len(not_matched)}):")
        print("-"*60)
        for idx, row in not_matched.iterrows():
            print(f"  - {row['Clean_Ticker']} (from: {row['Original_Ticker']})")
    
    return input_df

# Main execution
if __name__ == "__main__":
    print("="*60)
    print("TICKER COMPARISON TOOL")
    print("="*60)
    print("\nThis tool will:")
    print("  1. Clean ticker symbols (remove suffixes like 'UN Equity')")
    print("  2. Compare against reference file")
    print("  3. Create 3 output files:")
    print("     - Full results with match status")
    print("     - Matched tickers only")
    print("     - Non-matched tickers only")
    print("="*60)
    
    # Browse for input file
    print("\nPlease select your INPUT file (tickers to check)...")
    input_file = browse_file("Select INPUT file with tickers to check")
    
    if not input_file:
        print("No input file selected. Exiting.")
        exit()
    
    print(f"✓ Selected input file: {os.path.basename(input_file)}")
    
    # Browse for reference file
    print("\nPlease select your REFERENCE file (WLS index)...")
    reference_file = browse_file("Select REFERENCE file (WLS index)")
    
    if not reference_file:
        print("No reference file selected. Exiting.")
        exit()
    
    print(f"✓ Selected reference file: {os.path.basename(reference_file)}")
    
    # Run the comparison
    print("\nProcessing...")
    results = check_tickers(input_file, reference_file)
    
    print("\n" + "="*60)
    print("✓ PROCESS COMPLETE!")
    print("="*60)
    print("\nCheck the output files in the same directory where you ran this script.")

TICKER COMPARISON TOOL

This tool will:
  1. Clean ticker symbols (remove suffixes like 'UN Equity')
  2. Compare against reference file
  3. Create 3 output files:
     - Full results with match status
     - Matched tickers only
     - Non-matched tickers only

Please select your INPUT file (tickers to check)...
✓ Selected input file: Book1.csv

Please select your REFERENCE file (WLS index)...
✓ Selected reference file: Ticker Only BIO Tech WLS as of Oct 23 20251.csv

Processing...
Reading input file: Book1.csv
Reading reference file: Ticker Only BIO Tech WLS as of Oct 23 20251.csv
Input ticker column: Ticker
Reference ticker column: AAPG

SUMMARY
Total tickers in input file: 26
Tickers found in reference: 19 (73.1%)
Tickers NOT found in reference: 7 (26.9%)

FILES CREATED:
1. ticker_comparison_results.csv - Full comparison results
2. tickers_MATCHED.csv - Tickers found in reference (19 tickers)
3. tickers_NOT_MATCHED.csv - Tickers NOT in reference (7 tickers)

----------------------